<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Edit_NN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchの学習済みモデルを自由自在に書き換えたい

PyTorchには[PyTorch Image Models](https://github.com/rwightman/pytorch-image-models)など学習済モデルがたくさん公開されていて、これを転移学習に使うことも多いです。その際、学習済モデルを少し改造して試したい場合、どうすればいいのか。直接編集するわけではありませんが、同等の効果がある方法をご紹介します。

```
要点
1. 挿入するにはtorch.nn.Sequentialで置き換える
2. 削除するにはtorch.nn.Indentityで置き換える
```





まずはtorch.hubからお馴染みのresnet18を取得して、その構造を表示してみましょう。

In [ ]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)
print(model)

括弧で括られた部分がそのレイヤーの名前です。例えば(fc)となっている最終レイヤーにアクセスするには以下のようにします。

In [ ]:
print(model.fc)

ここで、出力の特徴数を変更するには以下です。

In [ ]:
model.fc.out_features = 100
print(model.fc)

同様にレイヤーを置き換えるのは簡単です。

In [ ]:
print('before:', model.relu)
model.relu = torch.nn.LeakyReLU(inplace=True)
print('after:', model.relu)

それではあるレイヤーを挿入したい場合はどうすればいいでしょうか。レイヤーを挿入するメソッドを見つけられなかったので、torch.nn.Sequentialを使って、以下のようにしました。以下はfc層の直前にDropoutを挿入する例です。

In [ ]:
from torch.nn import Sequential, Dropout
model.fc = Sequential(Dropout(0.5), model.fc)
print(model)

上手く挿入されました。

それではレイヤを削除したい場合はどうすればいいでしょうか。例えば最終層のクラス分類を外して、直前のレイヤーの出力を利用したいような場合です。この場合も、レイヤーを削除するメソッドが見つけられなかったので、以下のようにnn.Identityで置き換えます。

In [ ]:
model.fc = torch.nn.Identity()
print(model)

これで一つ前の(global_pool)の出力をそのまま出力するようにできたので、クラス分類層を削除したのと同様になりました。

ところで、モデルの構造をよく見ると、(0)のように示されているものがあります。これはmodel.layer1.0などではアクセスできませんので、以下のようにします。

In [ ]:
model.layer1[0]

無事にアクセスできました。これでどんな層も自由に編集できます。

以上です。